In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
import polars as pl
import polars_ts as pts

In [4]:
df = (
    pl.scan_parquet('https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet')
    .filter(pl.col("unique_id").is_in(["H1", "H2", "H3"]))
    .collect()
)
df

unique_id,ds,y
str,i64,f64
"""H1""",1,605.0
"""H1""",2,586.0
"""H1""",3,586.0
"""H1""",4,559.0
"""H1""",5,511.0
…,…,…
"""H3""",744,1791.0
"""H3""",745,1819.0
"""H3""",746,1791.0


In [5]:
from statsforecast import StatsForecast
from statsforecast.models import OptimizedTheta, AutoETS

season_length = 24

models = [
    OptimizedTheta(season_length=season_length, decomposition_type="additive"), # multiplicative   additive
    AutoETS(season_length=season_length)
]
sf = StatsForecast(models=models, freq=1, n_jobs=-1)
res = df.ts.kaboudan(sf, block_size=200, backtesting_start=0.5, n_folds=10)
res

OptimizedTheta,AutoETS
f64,f64
0.484785,0.066478
